# Welcome  

Notebook Author: Samuel Alter  
Notebook Subject: Capstone Project - Geographic Analysis

BrainStation Winter 2023: Data Science

## Introduction

This notebook walks through an analysis of geographic data (i.e., elevation and aspect) - looking at its influence on wildfire incidence. We want to see if these features of the landscape can accurately predict when a wildfire will be more likely. The location of study for this project are the Santa Monica Mountains, an east-west trending mountain range.

The dataset consists of a high-density grid that have elevation, and aspect data appended to each point. Also added to each point is whether there was a wildfire in that location. The elevation and aspect data are sourced from [USGS EarthExplorer](https://earthexplorer.usgs.gov), using the SRTM dataset. The wildfire data is sourced from the [National Interagency Fire Center](https://data-nifc.opendata.arcgis.com/datasets/nifc::interagencyfireperimeterhistory-all-years-view/explore?location=39.778749%2C-121.769073%2C11.96). 

Please refer to the visualization notebook to see maps of the field site.

## Initial Setup

### Imports

In [10]:
import pandas as pd
import numpy as np
import seaborn as sns

import statsmodels.api as sm

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [11]:
# import geographic information .CSV

sm_geo_original=pd.read_csv('/Users/sra/Desktop/Data_Science_2023/_capstone/00_capstone_data/shapefiles/joins/sm_geo_combine.csv')
sm_geo_original

,objectid,perim_id,perim_elevation_1,perim_asp_1,fire,geometry
0,4751.0,1644.0,434.0,31.883564,1,MULTIPOLYGON (((-118.297645979672 34.135057069...
1,4751.0,1645.0,349.0,7.624194,1,MULTIPOLYGON (((-118.297645979672 34.135057069...
2,4751.0,1646.0,288.0,72.613029,1,MULTIPOLYGON (((-118.297645979672 34.135057069...
3,4751.0,1647.0,189.0,335.170654,1,MULTIPOLYGON (((-118.297645979672 34.135057069...
4,4751.0,1648.0,229.0,346.464142,1,MULTIPOLYGON (((-118.297645979672 34.135057069...
...,...,...,...,...,...,...
19140,0.0,5935.0,52.0,149.036240,0,POINT (-118.2930000000007 34)
19141,0.0,5936.0,51.0,185.194427,0,POINT (-118.2880000000007 34)
19142,0.0,5937.0,52.0,270.000000,0,POINT (-118.2830000000007 34)
19143,0.0,5938.0,54.0,341.565063,0,POINT (-118.2780000000007 34)


### EDA

In [12]:
sm_geo_original.isna().sum()

objectid               0
perim_id             641
perim_elevation_1    641
perim_asp_1          668
fire                   0
geometry               0
dtype: int64

In [13]:
sm_geo_original[sm_geo_original['perim_asp_1'].isna()==True]

,objectid,perim_id,perim_elevation_1,perim_asp_1,fire,geometry
14,4753.0,NaN,NaN,NaN,1,MULTIPOLYGON (((-118.305363835672 34.119507991...
15,4767.0,NaN,NaN,NaN,1,MULTIPOLYGON (((-118.843533279074 34.169495387...
24,4796.0,NaN,NaN,NaN,1,MULTIPOLYGON (((-118.60574361788 34.1457095562...
25,4849.0,NaN,NaN,NaN,1,MULTIPOLYGON (((-118.377669263434 34.122895200...
26,4850.0,NaN,NaN,NaN,1,MULTIPOLYGON (((-118.574062124596 34.079625547...
...,...,...,...,...,...,...
18743,0.0,4600.0,32.0,NaN,0,POINT (-118.3680000000007 34.04)
18786,0.0,4752.0,53.0,NaN,0,POINT (-118.4330000000006 34.035)
18810,0.0,4776.0,67.0,NaN,0,POINT (-118.3130000000007 34.035)
18863,0.0,4941.0,52.0,NaN,0,POINT (-118.3130000000007 34.03)


In [14]:
(sm_geo_original.isna().sum())/(sm_geo_original.count())*100

objectid             0.000000
perim_id             3.464116
perim_elevation_1    3.464116
perim_asp_1          3.615306
fire                 0.000000
geometry             0.000000
dtype: float64

I have enough data that $3.5{\%}$ (roughly $650$ rows) is not a large portion of the dataset. I will remove these `NaN` rows:

In [15]:
sm_geo_original=sm_geo_original.dropna()
sm_geo_original.isna().sum()

objectid             0
perim_id             0
perim_elevation_1    0
perim_asp_1          0
fire                 0
geometry             0
dtype: int64

Convert aspect, which is a continuous variable, into a categorical.

First, rename columns.

In [16]:
# rename columns
sm_geo=sm_geo_original.copy()

sm_geo['asp_cont']=sm_geo['perim_asp_1']

sm_geo=sm_geo.drop(columns=(['perim_asp_1']))

sm_geo['elevation']=sm_geo['perim_elevation_1']

sm_geo=sm_geo.drop(columns=(['perim_elevation_1']))

sm_geo

,objectid,perim_id,fire,geometry,asp_cont,elevation
0,4751.0,1644.0,1,MULTIPOLYGON (((-118.297645979672 34.135057069...,31.883564,434.0
1,4751.0,1645.0,1,MULTIPOLYGON (((-118.297645979672 34.135057069...,7.624194,349.0
2,4751.0,1646.0,1,MULTIPOLYGON (((-118.297645979672 34.135057069...,72.613029,288.0
3,4751.0,1647.0,1,MULTIPOLYGON (((-118.297645979672 34.135057069...,335.170654,189.0
4,4751.0,1648.0,1,MULTIPOLYGON (((-118.297645979672 34.135057069...,346.464142,229.0
...,...,...,...,...,...,...
19140,0.0,5935.0,0,POINT (-118.2930000000007 34),149.036240,52.0
19141,0.0,5936.0,0,POINT (-118.2880000000007 34),185.194427,51.0
19142,0.0,5937.0,0,POINT (-118.2830000000007 34),270.000000,52.0
19143,0.0,5938.0,0,POINT (-118.2780000000007 34),341.565063,54.0


In [36]:
def degToCompass(num):
    '''
    From https://stackoverflow.com/questions/\
    7490660/converting-wind-direction-in-angles-to-text-words
    
    Takes a value between 0 and 360 and 
    outputs the corresponding compass representation
    '''
    val=int((num/22.5)+.5)
    arr=["N","NNE","NE","ENE","E","ESE", "SE", "SSE","S","SSW","SW","WSW","W","WNW","NW","NNW"]
    return arr[(val % 16)]

degToCompass(355)

'N'

In [39]:
sm_geo['asp_cont']

0         31.883564
1          7.624194
2         72.613029
3        335.170654
4        346.464142
            ...    
19140    149.036240
19141    185.194427
19142    270.000000
19143    341.565063
19144    270.000000
Name: asp_cont, Length: 18477, dtype: float64

In [40]:
# `.apply` the function degToCompass
# to each row in the column thusly:
sm_geo['asp_cat']=sm_geo['asp_cont'].apply(degToCompass)
sm_geo

,objectid,perim_id,fire,geometry,asp_cont,elevation,asp_cat
0,4751.0,1644.0,1,MULTIPOLYGON (((-118.297645979672 34.135057069...,31.883564,434.0,NNE
1,4751.0,1645.0,1,MULTIPOLYGON (((-118.297645979672 34.135057069...,7.624194,349.0,N
2,4751.0,1646.0,1,MULTIPOLYGON (((-118.297645979672 34.135057069...,72.613029,288.0,ENE
3,4751.0,1647.0,1,MULTIPOLYGON (((-118.297645979672 34.135057069...,335.170654,189.0,NNW
4,4751.0,1648.0,1,MULTIPOLYGON (((-118.297645979672 34.135057069...,346.464142,229.0,NNW
...,...,...,...,...,...,...,...
19140,0.0,5935.0,0,POINT (-118.2930000000007 34),149.036240,52.0,SSE
19141,0.0,5936.0,0,POINT (-118.2880000000007 34),185.194427,51.0,S
19142,0.0,5937.0,0,POINT (-118.2830000000007 34),270.000000,52.0,W
19143,0.0,5938.0,0,POINT (-118.2780000000007 34),341.565063,54.0,NNW


### Prepare data for analysis

In [19]:
X=sm_geo[['elevation','perim_asp_1']]
y=sm_geo[['fire']]

y

KeyError: "['perim_asp_1'] not in index"

In [ ]:
print(X.shape)
print(y.shape)

In [ ]:
# reshape the y data into a 1D array for modeling

y_array=np.ravel(a=y,order='C')
y_array

In [ ]:
y.sum()/y.count()

The dataset has $89{\%}$ of the landscape enduring a wildfire.

In [ ]:
X.corr()

Additionally, there is no collinearity between aspect and elevation.

## Modeling

### Train, Test, Split

Partition dataset to have $0.\overline3$ of the total as testing.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y_array, test_size=(1/3),stratify=y)

#### Basic logistic regression:

##### `sklearn`:

In [ ]:
# instantiate model
logreg=LogisticRegression()

# fit the model
logreg.fit(X_train,y_train)

print('Training Score: ',logreg.score(X_train,y_train))
print('Testing Score:  ',logreg.score(X_test,y_test))

Good match with the training set, though the accuracy is lower than I'd like; it's the same as the base rate. What about using `statsmodels` to see if we can make a stronger statistical understanding about what influences wildfires?

##### `statsmodels`:

In [ ]:
# add constant to prepare for statsmodels

X_withconstant=sm.add_constant(X)
X_withconstant.head()

In [ ]:
# instantiate model
logreg_sm=sm.Logit(y,X_withconstant)

# fit the model
logreg_sm_results=logreg_sm.fit()

# summary
logreg_sm_results.summary()

In [ ]:
logreg_sm_results.params

In [ ]:
beta0=logreg_sm_results.params[0] # constant
beta1=logreg_sm_results.params[1] # elevation
beta2=logreg_sm_results.params[2] # aspect

betas=[beta0,beta1,beta2]
odds=[]

for b in betas:
    odds.append(np.exp(b))
    
print(odds)

Now let's try to increase the accuracy of our model.